# MAHTS generation

## Background


## Description

* Take full time series of NDWI in dask
* Compute 2D tides for every timestep
* Compute median of tides from entire tide timeseries
* For each year in dask NDWI timeseries:
    * Mask pixels where tide > overall median
    * `.compute()` and take median 
    
***


## Getting started


### Load packages

First we import the required Python packages, then we connect to the database, and load the catalog of virtual products.

In [1]:
%matplotlib inline
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

import deacoastlines_generation as dcl_gen

import os
import sys
import datacube
import geopandas as gpd
from functools import partial

sys.path.append('../Scripts')
from dea_plotting import display_map

dc = datacube.Datacube(app='MAHTS_testing', env='c3-samples')

# from datacube.utils.dask import start_local_dask
# client = start_local_dask(mem_safety_margin='3gb')
# display(client)

import datetime
start_time = datetime.datetime.now()

### Load supplementary data

In [6]:
# Tide points are used to model tides across the extent of the satellite data
points_gdf = gpd.read_file('input_data/tide_points_coastal.geojson')

# Albers grid cells used to process the analysis
gridcell_gdf = (gpd.read_file('input_data/50km_albers_grid_clipped.shp')
            .to_crs(epsg=4326)
            .set_index('id'))

## Loading data
### Create query


In [8]:
study_area = 2585
output_name = 'nbar'

# If output folder doesn't exist, create it
output_dir = f'output_data/{study_area}_{output_name}'
os.makedirs(output_dir, exist_ok=True)
    
study_area_geopoly = dcl_gen.get_geopoly(study_area, gridcell_gdf)
query = {'geopolygon': study_area_geopoly,
         'time': ('1987', '2019'),
         'cloud_cover': [0, 90],
         'dask_chunks': {'time': 1, 'x': 1000, 'y': 1000}}

# Preview study area
display_map(x=(query['geopolygon'].envelope.left, 
               query['geopolygon'].envelope.right), 
            y=(query['geopolygon'].envelope.top, 
               query['geopolygon'].envelope.bottom))


### Load virtual product

In [10]:
# Load virtual product    
ds = dcl_gen.load_mndwi(dc, 
                        query, 
                        yaml_path='deacoastlines_virtual_products.yaml',
                        virtual_products=False)
ds

Loading ga_ls5t_ard_3 data
    Applying pixel quality/cloud mask
    Applying invalid data mask
Loading ga_ls7e_ard_3 data
    Applying pixel quality/cloud mask
    Applying invalid data mask
Loading ga_ls8c_ard_3 data
    Applying pixel quality/cloud mask
    Applying invalid data mask
Combining and sorting data
    Returning 1769 observations as a dask array
Dropping bands ['nbar_blue', 'nbar_green', 'nbar_red', 'nbar_nir', 'nbar_swir_1', 'nbar_swir_2']


<xarray.Dataset>
Dimensions:  (time: 1769, x: 1765, y: 1739)
Coordinates:
  * y        (y) float64 -1.743e+06 -1.743e+06 ... -1.795e+06 -1.795e+06
  * x        (x) float64 6.16e+05 6.16e+05 6.16e+05 ... 6.688e+05 6.689e+05
  * time     (time) datetime64[ns] 1987-05-19T01:06:26.795809 ... 2019-08-14T01:35:58.630949
Data variables:
    mndwi    (time, y, x) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
Attributes:
    crs:      epsg:32651

## Tidal modelling
### Model tides at point locations

In [ ]:
tidepoints_gdf = dcl_gen.model_tides(ds, points_gdf)

In [ ]:
import matplotlib.pyplot as plt

tidepoints_gdf.plot()
gpd.GeoDataFrame(crs=gridcell_gdf.crs, 
                 geometry=[gridcell_gdf.loc[study_area].geometry]).plot(ax=plt.gca())

In [ ]:
study_area_gdf = gridcell_gdf.loc[[study_area]]
buffered = study_area_gdf.buffer(0.075).unary_union
subset_gdf = points_gdf[points_gdf.geometry.intersects(buffered)]



In [ ]:
study_area_gdf.buffer(0.05).plot()
subset_gdf.plot(ax=plt.gca())

In [ ]:
subset_gdf.shape

In [ ]:
from shapely.geometry import shape
ds_extent = shape(ds.geobox.geographic_extent.json)
buffered = ds_extent.buffer(0.025)
subset_gdf = points_gdf[points_gdf.geometry.intersects(buffered)]

In [ ]:
points_gdf[points_gdf.intersects(study_area_gdf.buffer(0.1).unary_union)]

In [ ]:
study_area_gdf.buffer(0.1)

In [ ]:
points_gdf

In [ ]:
subset_gdf = points_gdf[points_gdf.geometry.intersects(study_area_gdf.geometry)]

In [ ]:
study_area_gdf.plot()

### Interpolate tides into each satellite timestep

In [ ]:
# Interpolate tides for each timestep into the spatial extent of the data
interp_tide = partial(dcl_gen.interpolate_tide,
                      tidepoints_gdf=tidepoints_gdf,
                      factor=50, dask=False)
ds['tide_m'] = dcl_gen.multiprocess_apply(ds=ds,
                                          dim='time',
                                          func=interp_tide)

In [ ]:
# interp_tide = partial(dcl_gen.interpolate_tide,
#                       tidepoints_gdf=tidepoints_gdf,
#                       factor=50, dask=True)
# ds['tide_m'] = ds.groupby('time').apply(interp_tide)

In [ ]:
import matplotlib.pyplot as plt

# Plot 
ds_i = ds['tide_m'].isel(time=50).compute()
ds_i.plot.imshow(robust=True, 
                 cmap='viridis', 
                 size=12, 
                 vmin=ds_i.min().item(), 
                 vmax=ds_i.max().item())
tidepoints_gdf.loc[str(ds_i.time.values)[0:10]].plot(ax=plt.gca(), 
                                                     column='tide_m', 
                                                     cmap='viridis', 
                                                     markersize=100,
                                                     edgecolor='black',
                                                     vmin=ds_i.min().item(), 
                                                     vmax=ds_i.max().item())

In [ ]:
# Determine tide cutoff
tide_cutoff_buff = (
    (ds['tide_m'].max(dim='time') - ds['tide_m'].min(dim='time')) * 0.25)
tide_cutoff_min = 0.0 - tide_cutoff_buff
tide_cutoff_max = 0.0 + tide_cutoff_buff

## Generate yearly composites

In [ ]:
# Iterate through each year and export annual and 3-year gapfill composites
dcl_gen.export_annual_gapfill(ds, 
                              output_dir, 
                              tide_cutoff_min, 
                              tide_cutoff_max)

In [ ]:
print(f'{(datetime.datetime.now() - start_time).seconds / 60:.1f} minutes')

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** March 2020